In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/'My Drive'/doan

/content/drive/My Drive/doan


**IMPORT DATA**

In [4]:
image_size = (64, 64)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Class_Human",
    validation_split=0.2,
    subset="training",
    seed=18,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Class_Human",
    validation_split=0.2,
    subset="validation",
    seed=18,
    image_size=image_size,
    batch_size=batch_size,
)


Found 21758 files belonging to 2 classes.
Using 17407 files for training.
Found 21758 files belonging to 2 classes.
Using 4351 files for validation.


In [5]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

**TRAINNING**

Model

In [6]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,Rescaling
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from keras.models import Sequential

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard

In [8]:
optimizer = tf.keras.optimizers.Adam()
callbacks = [
          #Luu bo trong so weight    
          ModelCheckpoint("files/model.h5"),
          #Theo doi loss function va giam LR
          #factor gia tri learning rate se giam 
          #patience: sau so chu ky ma LR se giam
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3),
          #Truyen lai ket qua vao CSV
          CSVLogger("files/data.csv"),
          TensorBoard(),
          #Theo doi loss function
          #patience: sau so chu ky neu khong cai thien thi dung
          #restore_best_weights: lua chon khoi phuc tu epoch co ket qua tot nhat
          EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
]

In [9]:
input_shape = (64, 64, 3)


model = Sequential()
model.add(Conv2D(16, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1_conv1',input_shape=(64, 64, 3)))
model.add(Conv2D(16, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block1_maxpool'))

model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block2_conv1'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block2_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block2_maxpool'))

model.add(Conv2D(32, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv1'))
model.add(Conv2D(32, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv2'))
model.add(Conv2D(32, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv3'))
model.add(Conv2D(32, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv4'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block3_maxpool'))

model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv1'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv2'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv3'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv4'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block4_maxpool'))

model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv1'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv2'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv3'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv4'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block5_maxpool'))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

def optimizer_init_fn(): 
  learning_rate = 1e-4
  return tf.keras.optimizers.Adam(learning_rate) 

model.compile(optimizer=optimizer_init_fn(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fit model (training)
history = model.fit(train_ds, steps_per_epoch=len(train_ds),
    validation_data=val_ds, validation_steps=len(val_ds), epochs=30, verbose=1,callbacks=callbacks)

Epoch 1/30
544/544 [==============================] - 1541s 3s/step - loss: 0.5172 - accuracy: 0.8664 - val_loss: 0.1405 - val_accuracy: 0.9451 - lr: 1.0000e-04
Epoch 2/30
544/544 [==============================] - 46s 83ms/step - loss: 0.1401 - accuracy: 0.9472 - val_loss: 0.1422 - val_accuracy: 0.9487 - lr: 1.0000e-04
Epoch 3/30
544/544 [==============================] - 50s 90ms/step - loss: 0.1086 - accuracy: 0.9592 - val_loss: 0.0784 - val_accuracy: 0.9687 - lr: 1.0000e-04
Epoch 4/30
544/544 [==============================] - 46s 84ms/step - loss: 0.0759 - accuracy: 0.9720 - val_loss: 0.0662 - val_accuracy: 0.9770 - lr: 1.0000e-04
Epoch 5/30
544/544 [==============================] - 46s 84ms/step - loss: 0.0574 - accuracy: 0.9802 - val_loss: 0.0784 - val_accuracy: 0.9726 - lr: 1.0000e-04
Epoch 6/30
544/544 [==============================] - 45s 83ms/step - loss: 0.0496 - accuracy: 0.9825 - val_loss: 0.1575 - val_accuracy: 0.9494 - lr: 1.0000e-04
Epoch 7/30
544/544 [==============